In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium==4.26.1

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,223 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,323 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

In [5]:
# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In [3]:
!python --version

import selenium
print("selenium",selenium.__version__)

Python 3.10.12
selenium 4.26.1


In [6]:
# Import modules
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from google.colab import drive
import os
import json

In [7]:
# Install the correct version of chromedriver
chromedriver_autoinstaller.install()

In [8]:
# Set up the Chrome driver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [29]:
driver = webdriver.Chrome(options=options)

In [10]:
# Open the YouTube channel's videos page
driver.get("https://www.youtube.com/@kamalaharris/videos")

In [11]:
# Scroll and collect hrefs
SCROLL_PAUSE_TIME = 2
NUM_VIDEOS_TO_COLLECT = 150

In [12]:
hrefs = []
last_height = driver.execute_script("return document.documentElement.scrollHeight")

In [13]:
video_data = []
while len(video_data) < NUM_VIDEOS_TO_COLLECT:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait for new content to load
    time.sleep(SCROLL_PAUSE_TIME)

    # Find all video elements
    elements = driver.find_elements(By.CSS_SELECTOR, 'ytd-rich-grid-media')
    for element in elements:
        # Extract title
        title_element = element.find_element(By.CSS_SELECTOR, 'yt-formatted-string#video-title')
        title = title_element.text

        # Extract URL
        link_element = element.find_element(By.CSS_SELECTOR, 'a.yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail')
        href = link_element.get_attribute('href')

        # Add to video data if unique
        if href and not any(video['url'] == href for video in video_data):
            video_data.append({
                'title': title,
                'url': href
            })

    # Check if we've reached the bottom of the page
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [14]:
video_data

[{'title': '64,000 | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=L2HRvz1vvc8'},
 {'title': 'Girl Dad | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=2x72eSfZ7Ao'},
 {'title': 'Oprah Winfrey Welcomes First-Time Voters To A Kamala Harris Rally',
  'url': 'https://www.youtube.com/watch?v=RR4nUCMvXFY'},
 {'title': 'Big Things | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=eGPCopBNieM'},
 {'title': 'Good Sense | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=_v_4IgeVQ8M'},
 {'title': 'This Time | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=t9suRUZGIdg'},
 {'title': 'Promise | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=OwFWgewAAXc'},
 {'title': 'Different | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=Cb0AothNHiw'},
 {'title': 'We Rise | Harris-Walz 2024',
  'url': 'https://www.youtube.com/watch?v=oU9IVXyGBYI'},
 {'title': 'Mother | Narrated by Viola Davis for Kamala Harris',
  'u

In [17]:
len(video_data)

150

In [15]:
# Prepare save path
save_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube'
os.makedirs(save_path, exist_ok=True)
file_path = os.path.join(save_path, 'harris_youtube_url_title.json')

In [16]:
# Save the video data to a JSON file
with open(file_path, 'w') as f:
    json.dump(video_data, f, ensure_ascii=False, indent=4)

In [ ]:
# 7월 21부터 최근까지 영상개수가 150개 이하로 추정
# 정확한 날짜로 필터링 하기 위해서는 해당 url을 모두 접속하여 업로드 날짜를 스크래핑하고 필터링하도록 한다.

In [18]:
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/harris_youtube_url_title.json', 'r') as f:
    urls = json.load(f)

In [39]:
# Selenium WebDriver 설정
driver = webdriver.Chrome(options=options)  # Chrome 드라이버 초기화
wait = WebDriverWait(driver, 10)

In [40]:
dates = []

for item in urls:
    url = item['url']  # 'url' 키를 통해 URL 추출
    driver.get(url)  # 올바르게 URL 호출

    # 페이지 로딩 대기
    time.sleep(2)  # 추가 대기 시간

    # "설명" 섹션 클릭
    try:
        description_div = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div#description-inner')))
        driver.execute_script("arguments[0].click();", description_div)
        time.sleep(1)  # 클릭 후 대기
    except Exception as e:
        print(f'"설명" 섹션을 클릭할 수 없습니다: {url}')
        print(f'에러: {e}')
        continue  # 다음 URL로 진행

    # 날짜 스크래핑
    try:
        # 모든 <span> 태그를 선택
        span_elements = driver.find_elements(By.CSS_SELECTOR, 'yt-formatted-string#info span.style-scope.bold')

        # 두 번째 <span> 태그의 텍스트 가져오기
        if len(span_elements) > 1:  # 세 번째 <span>이 있는지 확인
            date_text = span_elements[2].text.strip()
            dates.append({'url': url, 'date': date_text})
            print(f'날짜를 찾았습니다: {date_text}')
        else:
            print(f'날짜를 찾을 수 없습니다: {url}')
    except Exception as e:
        print(f'날짜를 찾을 수 없습니다: {url}')
        print(f'에러: {e}')

날짜를 찾았습니다: Nov 5, 2024
날짜를 찾았습니다: Nov 5, 2024
날짜를 찾았습니다: Nov 5, 2024
날짜를 찾았습니다: Nov 5, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Nov 3, 2024
날짜를 찾았습니다: Nov 3, 2024
날짜를 찾았습니다: Nov 2, 2024
날짜를 찾았습니다: Nov 2, 2024
날짜를 찾았습니다: Nov 2, 2024
날짜를 찾았습니다: Nov 1, 2024
날짜를 찾았습니다: Nov 1, 2024
날짜를 찾았습니다: Nov 1, 2024
날짜를 찾았습니다: Oct 31, 2024
날짜를 찾았습니다: Oct 31, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 29, 2024
날짜를 찾았습니다: Oct 28, 2024
날짜를 찾았습니다: Oct 28, 2024
날짜를 찾았습니다: Oct 28, 2024
날짜를 찾았습니다: Oct 28, 2024
날짜를 찾았습니다: Oct 28, 2024
날짜를 찾았습니다: Oct 28, 2024
날짜를 찾았습니다: Oct 27, 2024
날짜를 찾았습니다: Oct 27, 2024
날짜를 찾았습니다: Oct 26, 2024
날짜를 찾았습니다: Oct 26, 2024
날짜를 찾았습니다: Oct 26, 2024
날짜를 찾았습니다: Oct 25, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다:

In [41]:
dates

[{'url': 'https://www.youtube.com/watch?v=L2HRvz1vvc8', 'date': 'Nov 5, 2024'},
 {'url': 'https://www.youtube.com/watch?v=2x72eSfZ7Ao', 'date': 'Nov 5, 2024'},
 {'url': 'https://www.youtube.com/watch?v=RR4nUCMvXFY', 'date': 'Nov 5, 2024'},
 {'url': 'https://www.youtube.com/watch?v=eGPCopBNieM', 'date': 'Nov 5, 2024'},
 {'url': 'https://www.youtube.com/watch?v=_v_4IgeVQ8M', 'date': 'Nov 4, 2024'},
 {'url': 'https://www.youtube.com/watch?v=t9suRUZGIdg', 'date': 'Nov 4, 2024'},
 {'url': 'https://www.youtube.com/watch?v=OwFWgewAAXc', 'date': 'Nov 4, 2024'},
 {'url': 'https://www.youtube.com/watch?v=Cb0AothNHiw', 'date': 'Nov 4, 2024'},
 {'url': 'https://www.youtube.com/watch?v=oU9IVXyGBYI', 'date': 'Nov 4, 2024'},
 {'url': 'https://www.youtube.com/watch?v=ShwoUTax4B0', 'date': 'Nov 4, 2024'},
 {'url': 'https://www.youtube.com/watch?v=icMONoEE61M', 'date': 'Nov 3, 2024'},
 {'url': 'https://www.youtube.com/watch?v=Wo7KwWIg-7o', 'date': 'Nov 3, 2024'},
 {'url': 'https://www.youtube.com/watch?

In [43]:
# 결과를 JSON 파일로 저장
output_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube/harris_youtube_url_date.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(dates, f, ensure_ascii=False, indent=4)

In [44]:
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/harris_youtube_url_date.json', 'r') as f:
    urls = json.load(f)

In [47]:
len(urls)

150

In [45]:
from datetime import datetime

In [46]:
# Define the date range
start_date = datetime.strptime("Jul 21, 2024", "%b %d, %Y")
end_date = datetime.strptime("Nov 4, 2024", "%b %d, %Y")

In [48]:
# Function to parse the date and check if it falls within the desired range
def is_within_range(date_str):
    # Handle "Premiered" date format
    if "Premiered" in date_str:
        date_str = date_str.replace("Premiered", "").strip()

    # Try parsing the date
    try:
        # Try multiple formats for flexibility
        date_obj = datetime.strptime(date_str, "%b %d, %Y")
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, "%b %d, %Y")
        except ValueError:
            return False

    # Check if the date is within the desired range
    return start_date <= date_obj <= end_date

In [49]:
# Filter the data
filtered_urls = [url for url in urls if is_within_range(url['date'])]

In [50]:
len(filtered_urls)

126

In [51]:
# Save the filtered data back to the file
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/harris_youtube_url_title_date_filtered.json', 'w') as f:
    json.dump(filtered_urls, f, indent=4)

In [ ]:
# driver.quit()

In [ ]:
# title을 깜빡 잊어서 title 추가 후 수정

In [53]:
# Load harris_youtube_url_title_date_filtered.json
with open("/content/drive/MyDrive/[인공지능기초]/sns/youtube/harris_youtube_url_title_date_filtered.json", "r") as date_file:
    date_data = json.load(date_file)

# Load harris_youtube_url_title.json
with open("/content/drive/MyDrive/[인공지능기초]/sns/youtube/harris_youtube_url_title.json", "r") as title_file:
    title_data = json.load(title_file)

# Create a dictionary from title_data for quick lookup by URL
title_dict = {item["url"]: item["title"] for item in title_data}

# Add titles to date_data if the URL matches
for item in date_data:
    if item["url"] in title_dict:
        item["title"] = title_dict[item["url"]]

# Save the updated date_data to a new file
with open("harris_youtube_url_date_filtered_with_titles.json", "w") as output_file:
    json.dump(date_data, output_file, ensure_ascii=False, indent=4)

In [54]:
with open("/content/harris_youtube_url_date_filtered_with_titles.json", "r") as date_file:
    su_data = json.load(date_file)

In [55]:
len(su_data)

126

In [56]:
su_data

[{'url': 'https://www.youtube.com/watch?v=_v_4IgeVQ8M',
  'date': 'Nov 4, 2024',
  'title': 'Good Sense | Harris-Walz 2024'},
 {'url': 'https://www.youtube.com/watch?v=t9suRUZGIdg',
  'date': 'Nov 4, 2024',
  'title': 'This Time | Harris-Walz 2024'},
 {'url': 'https://www.youtube.com/watch?v=OwFWgewAAXc',
  'date': 'Nov 4, 2024',
  'title': 'Promise | Harris-Walz 2024'},
 {'url': 'https://www.youtube.com/watch?v=Cb0AothNHiw',
  'date': 'Nov 4, 2024',
  'title': 'Different | Harris-Walz 2024'},
 {'url': 'https://www.youtube.com/watch?v=oU9IVXyGBYI',
  'date': 'Nov 4, 2024',
  'title': 'We Rise | Harris-Walz 2024'},
 {'url': 'https://www.youtube.com/watch?v=ShwoUTax4B0',
  'date': 'Nov 4, 2024',
  'title': 'Mother | Narrated by Viola Davis for Kamala Harris'},
 {'url': 'https://www.youtube.com/watch?v=icMONoEE61M',
  'date': 'Nov 3, 2024',
  'title': "Election Night Preview with Kamala Harris' Campaign Chair Jen O'Malley Dillon"},
 {'url': 'https://www.youtube.com/watch?v=Wo7KwWIg-7o',
 